In [12]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Pulling down the Listing Data

In [20]:
r = requests.get('https://www.propertypro.ng/property-for-rent/flat-apartment/?search=&bedroom=&min_price=&max_price=3000000')

soup = BeautifulSoup(r.content, 'html5lib')

In [21]:
listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
print("Number of apartment listings on a page:", len(listing_divs))

Number of apartment listings on a page: 20


In [22]:
listing_divs[0]

<div class="single-room-sale listings-property">
<div class="single-room-img">
<a class="single-room-img-a" href="/property/3-bedroom-flat-apartment-for-rent-olaleye-estate-alaka-iponri-surulere-lagos-9ECUX"> <img alt="3 bedroom Flat / Apartment for rent OLALEYE Estate  Alaka/Iponri Surulere Lagos" class="listings-img" src="https://images.propertypro.ng/medium/brand-new-spacious-and-serviced-3bedroom-apartments-in-surulere-CfyqC5lSDiANDwxcT4gO.jpg" title="3 bedroom Flat / Apartment for rent OLALEYE Estate  Alaka/Iponri Surulere Lagos"/> </a>
<div class="save-heart">
<a href="/profile/add-favorite/574634"><i class="fa fa-heart"></i></a>
</div>
<div class="room-icons">
<span><span>9</span> <img alt="icon" src="/assets/assets/img/property/2f8b655773948c43f77e8c928e94bbe3-img.png"/> </span>
</div>

</div>
<div class="single-room-text">
<a href="/property/3-bedroom-flat-apartment-for-rent-olaleye-estate-alaka-iponri-surulere-lagos-9ECUX"> <h2 class="listings-property-title">3 BEDROOM FLAT /

In [23]:
listing_divs[0].select('h4')[0].text

'OLALEYE Estate Alaka Iponri Surulere Lagos'

In [24]:
listing_divs[0].select('h3[class*=listings-price]')[0].text.strip()

'₦ 2,500,000'

In [25]:
listing_divs[0].select('div[class*=fur-areea]')[0].text.strip().split('\n')

['3 beds', '4 baths', '4 Toilets']

In [51]:
listing_divs[7].select('div[class*=furnished-btn]')[0].text.strip().split('\n')

['Serviced', 'Newly Built']

Now that I've gotten all the relevant information, I'll put all the codes together;

In [53]:
all_listings_data = []
for indv_listing in listing_divs:
    address = indv_listing.select('h4')[0].text
    price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
    listing_data = [address, price]
    utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
    listing_data.extend(utilities)
    extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().split('\n')
    listing_data.append(extra)
    all_listings_data.append(listing_data)
    
pd.DataFrame(all_listings_data)

,0,1,2,3,4,5
0,OLALEYE Estate Alaka Iponri Surulere Lagos,"₦ 2,500,000",3 beds,4 baths,4 Toilets,[]
1,Ikota Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,[Newly Built]
2,"Off Kudirat Abiola way, Julie EstateOregun Ike...","₦ 2,500,000",3 beds,3 baths,4 Toilets,[]
3,Ikate Lekki Lagos,"₦ 3,000,000",3 beds,4 baths,4 Toilets,[Serviced]
4,ShimawaMowe Obafemi Owode Ogun,"₦ 260,000",3 beds,1 baths,2 Toilets,[Newly Built]
5,Lekki Phase 1 Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,[]
6,Lekki Phase 1 Lekki Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,[]
7,-Old Ikoyi Ikoyi Lagos,"$ 40,000",3 beds,3 baths,4 Toilets,"[Serviced, Newly Built]"
8,Oniru Victoria Island Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,[Serviced]
9,Ologolo Lekki Lagos,"₦ 2,600,000",3 beds,3 baths,4 Toilets,"[Serviced, Newly Built]"


**Writing a dynamic function to get the data;**

In [72]:
def parse_listing_data(location, max_price, rows_of_data=2000):
    all_listings_data = []
    page_num = 0
    
    while page_num < (rows_of_data / 20):
        url = "https://www.propertypro.ng/property-for-rent/flat-apartment/?search=" + location \
              + "&bedroom=&min_price=&max_price=" + str(max_price) + "&page=" + str(page_num)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib')
        listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
                
        for indv_listing in listing_divs:
            address = indv_listing.select('h4')[0].text
            price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
            listing_data = [address, price]
            utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
            listing_data.extend(utilities)
            extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().split('\n')
            listing_data.append(extra)
            all_listings_data.append(listing_data)
        page_num += 1    
    listing_df = pd.DataFrame(all_listings_data, columns=['neighborhood', 'price', 'beds', 'baths', 'toilets', 'extra'])
    return listing_df

### Getting Lagos' Listing Data

In [60]:
lagos_data = parse_listing_data('lagos', 2000000, 3000)

*************************1 page(s) done!******************************
*************************2 page(s) done!******************************
*************************3 page(s) done!******************************
*************************4 page(s) done!******************************
*************************5 page(s) done!******************************
*************************6 page(s) done!******************************
*************************7 page(s) done!******************************
*************************8 page(s) done!******************************
*************************9 page(s) done!******************************
*************************10 page(s) done!******************************
*************************11 page(s) done!******************************
*************************12 page(s) done!******************************
*************************13 page(s) done!******************************
*************************14 page(s) done!******************************
*

In [61]:
lagos_data

,neighborhood,price,beds,baths,toilets,extra
0,-Old Ikoyi Ikoyi Lagos,"$ 40,000",3 beds,3 baths,4 Toilets,"[Serviced, Newly Built]"
1,...Lekki Phase 1 Lekki Lagos,"₦ 800,000",1 beds,1 baths,1 Toilets,[]
2,Lekki Phase 1 Lekki Lagos,"₦ 1,200,000",1 beds,1 baths,1 Toilets,[]
3,St.Finbars Akoka Yaba Lagos,"₦ 500,000",beds,baths,1 Toilets,"[Furnished, Serviced, Newly Built]"
4,St. Finbars Akoka Yaba Lagos,"₦ 600,000",beds,baths,1 Toilets,"[Furnished, Serviced, Newly Built]"
...,...,...,...,...,...,...
2995,Abiola Estate Ayobo LagosAyobo Ipaja Lagos,"₦ 400,000",2 beds,2 baths,2 Toilets,[]
2996,Close to adekunle Ebute Metta Yaba Lagos,"₦ 300,000",1 beds,0 baths,0 Toilets,[]
2997,Iyana ipaja Ipaja Road Ipaja Lagos,"₦ 250,000",1 beds,1 baths,2 Toilets,[]
2998,MorrocoAbule Ijesha Yaba Lagos,"₦ 850,000",3 beds,2 baths,Toilets,[]


In [64]:
lagos_data.to_csv("rent_listings_lagos.csv", index=False)

### Getting Ibadan's Data

In [75]:
ibadan_data = parse_listing_data('oyo', 2500000, 2000)

In [76]:
ibadan_data

,neighborhood,price,beds,baths,toilets,extra
0,Iyaganku GRA quarters IbadanIyanganku Ibadan Oyo,"₦ 1,000,000",3 beds,3 baths,3 Toilets,[]
1,Behind DSS ile tuntun nihort Jericho ibadanIdi...,"₦ 450,000",3 beds,3 baths,3 Toilets,[]
2,Yidi apete junction ibadan Ibadan Polytechnic...,"₦ 126,000",1 beds,1 baths,1 Toilets,"[Serviced, Newly Built]"
3,"Adeoyo GRA, off ring road, ibadan Ring Rd Ibad...","₦ 750,000",3 beds,3 baths,4 Toilets,[Newly Built]
4,"Opposition nnpc filling station, akala express...","₦ 600,000",3 beds,3 baths,4 Toilets,[Newly Built]
...,...,...,...,...,...,...
506,"Salute, tipper. Akala express, ibadanAkala Exp...","₦ 200,000",1 beds,1 baths,2 Toilets,[Newly Built]
507,OrogunIbadan Oyo,"₦ 180,000",3 beds,2 baths,2 Toilets,[]
508,Elebu OjaAkala Express Ibadan Oyo,"₦ 350,000",2 beds,2 baths,2 Toilets,"[Furnished, Newly Built]"
509,Onigbodogi Apete IbadanAjibode Ibadan Oyo,"₦ 120,000",1 beds,1 baths,1 Toilets,[]


In [77]:
ibadan_data.to_csv("rent_listings_oyo.csv", index=False)

### Getting Abuja's Data

In [84]:
abuja_data = parse_listing_data('abuja', 3000000, 3000)

In [85]:
abuja_data.to_csv("rent_listings_abuja.csv", index=False)
abuja_data

,neighborhood,price,beds,baths,toilets,extra
0,DawakiGwarinpa Abuja,"₦ 1,500,000",3 beds,3 baths,4 Toilets,[Newly Built]
1,Gaduwa Abuja,"₦ 1,500,000",3 beds,3 baths,4 Toilets,[]
2,Maitama Abuja,"₦ 2,500,000",2 beds,2 baths,4 Toilets,[Serviced]
3,In an estate behind games village Garki 1 Abuja,"₦ 1,500,000",2 beds,2 baths,3 Toilets,[]
4,Dawaki opposite gwarinpaGwarinpa Abuja,"₦ 1,000,000",2 beds,2 baths,3 Toilets,[Newly Built]
...,...,...,...,...,...,...
677,Off 69 Road GwarimpaGwarinpa Abuja,"₦ 2,000,000",3 beds,3 baths,4 Toilets,"[Serviced, Newly Built]"
678,New site estate sauka close to FHALugbe Abuja,"₦ 550,000",1 beds,1 baths,1 Toilets,[Newly Built]
679,Kado estateKado Abuja,"₦ 1,500,000",2 beds,2 baths,3 Toilets,[Furnished]
680,...Durumi Abuja,"₦ 2,800,000",0 beds,0 baths,0 Toilets,[]


### Getting Ogun's Data

In [80]:
ogun_data = parse_listing_data('ogun', 1500000, 2000)

In [81]:
ogun_data.to_csv("rent_listings_ogun.csv", index=False)
ogun_data

,neighborhood,price,beds,baths,toilets,extra
0,ShimawaMowe Obafemi Owode Ogun,"₦ 260,000",3 beds,1 baths,2 Toilets,[Newly Built]
1,Adigbe Abeokuta Ogun,"₦ 300,000",2 beds,3 baths,3 Toilets,[]
2,Private EstateMagboro Obafemi Owode Ogun,"₦ 250,000",1 beds,1 baths,2 Toilets,[]
3,MagboroMagboro Obafemi Owode Ogun,"₦ 350,000",2 beds,2 baths,2 Toilets,[]
4,Private EstateArepo Ogun,"₦ 450,000",2 beds,2 baths,3 Toilets,[]
...,...,...,...,...,...,...
183,"Ima Road, Abiola Way, Abeokuta.Abeokuta Ogun","₦ 250,000",2 beds,2 baths,3 Toilets,[]
184,Egan road Iyesi otaAdo Odo Ota Ogun,"₦ 180,000",3 beds,4 baths,4 Toilets,"[Furnished, Newly Built]"
185,Abeokuta Street Dalemo Sango otaAdalemo Ado Od...,"₦ 200,000",1 beds,1 baths,1 Toilets,[]
186,Ibafo Obafemi Owode Ogun,"₦ 350,000",2 beds,2 baths,2 Toilets,[Newly Built]


### Getting PortHarcourt's Data

In [82]:
ph_data = parse_listing_data('rivers', 2500000, 2500)

In [86]:
ph_data.to_csv("rent_listings_ph.csv", index=False)
ph_data

,neighborhood,price,beds,baths,toilets,extra
0,GRA phase 3 tombia extensionNew Gra Port Harco...,"₦ 1,500,000",2 beds,2 baths,2 Toilets,[Newly Built]
1,Ada George Port Harcourt Rivers,"₦ 500,000",2 beds,2 baths,2 Toilets,"[Furnished, Newly Built]"
2,odili roadTrans Amadi Port Harcourt Rivers,"₦ 550,000",1 beds,1 baths,2 Toilets,[]
3,Trans Amadi Port Harcourt Rivers,"₦ 650,000",beds,baths,Toilets,[]
4,Trans Amadi Port Harcourt Rivers,"₦ 700,000",beds,baths,Toilets,[]
...,...,...,...,...,...,...
99,Woji Port Harcourt Rivers,"₦ 750,000",1 beds,2 baths,2 Toilets,"[Serviced, Newly Built]"
100,RumuodaraPort Harcourt Rivers,"₦ 800,000",3 beds,4 baths,4 Toilets,[]
101,Off peter odili roadTrans Amadi Port Harcourt ...,"₦ 1,400,000",3 beds,4 baths,4 Toilets,[Newly Built]
102,"N0.3 DR.ANJU CLOSE, OFF CHIEF UMEH STREET, AGI...","₦ 800,000",3 beds,2 baths,2 Toilets,[]
